# Primera sesion practica: manejo de anotaciones

### Primera sesión: anotación de grabaciones y manejo de anotaciones 

1. Para este ejercicio vamos a hacer 'strong labels' sobre una grabación desconocida. Para esto vamos a usar el software Raven lite 2. Para descargarlo solo es necesario ir a [esta pagina](https://www.ravensoundsoftware.com/raven-lite-downloads/) y descargar la version según su dispositivo.

2. Crea un ambiente virtual: Para esta practica es necesario instalar el paquete Opensoundscape, debido a que este tiene muchas dependencias asociadas que pueden entrar en conflicto facilmente, es recomendable crear un ambiente virtual individual, para ello vamos a usar conda, luego vamos a descargar la version mas reciente de opensoundscape. Hay otras formas de instalar este paquete detalladas en la [documentacion](https://opensoundscape.org/en/latest/installation/mac_and_linux.html)

```
conda create --name opensoundscape python==3.10
conda activate opensoundscape
pip install opensoundscape==0.12.0
```
3. Descarga los datos para este ejercicio: descarga los archivos `annotation_Files.zip` y `mp3_Files.zip` del este [link](https://datadryad.org/dataset/doi:10.5061/dryad.d2547d81z). Mueve los archivos descomprimidos a la carpeta "/AI_Andes/Sesiones_practicas/Primera_practica/datos_anotados"

In [43]:
# Paquetes necesarios
from opensoundscape import Audio, Spectrogram
from opensoundscape.annotations import BoxedAnnotations
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path
from matplotlib import pyplot as plt
import os
plt.rcParams['figure.figsize']=[15,5]
%config InlineBackend.figure_format = 'retina'

## Estructura de los datos

Hay varias formas de guardar anotaciones, con el objetivo de no generar archivos duplicados durante todo el flujo de trabajo lo mas eficiente es mantener los archivos de anotacion archivos de anotacion en txt que estan relacionados con una grabacion mas extensa. 

In [ ]:
# Asegurese que este path relativo sea identificado correctamente en su ordenador
dataset_path = Path("./datos_anotados/")

# Cambio de nombre de una de las carpetas
if os.path.exists('./datos_anotados/mp3_Files'):
    os.rename('./datos_anotados/mp3_Files', './datos_anotados/Recordings')
else:
    print('Carpeta correctamente renombrada')

# Ejemplo con un archivo de audio y el correspondiente archivo de anotación de Raven
audio_file = "./datos_anotados/Recordings/Recording_2/Recording_2_Segment_01.mp3"
annotation_file = "./datos_anotados/annotation_Files/Recording_2/Recording_2_Segment_01.Table.1.selections.txt"
annotation_table = pd.read_csv(annotation_file, sep='\t')

# Despleguemos un espectrograma y una parte de la tabla con sus anotaciones 
Spectrogram.from_audio(Audio.from_file(audio_file)).plot()
print(annotation_table.head(10))


## Organizar las "Cajas" con anotaciones 

In [59]:
# Hacemos una lista con los archivos de texto con las anotaciones
selections = glob(f"{dataset_path}/Annotation_Files/*/*.txt")

# Creamos una lista de archivos de audio, uno correspondiente a cada archivo de Raven
# (Los archivos de audio tienen los mismos nombres que las tablas de Raven con diferente extension)
audio_files = [
    f.replace("Annotation_Files", "Recordings").replace(
        ".Table.1.selections.txt", ".mp3"
    )
    for f in selections
]

Luego de "conectar" los archivos de audio con las anotaciones al cambiar los paths, ahora solo es necesario crear una tabla que incluya la informacion de cada caja junto con el path del archivo de sonido correspondiente.

In [ ]:
annotations = BoxedAnnotations.from_raven_files(
    selections, annotation_column="Species", # Recuerde especificar la identidad de cada caja (la unica columna disponible en Raven lite)
    audio_files=audio_files
)
annotations.df.head(5)

In [ ]:
# Miremos cuantas grabaciones por clase tenemos
annotations.df.annotation.value_counts()

In [ ]:
# OpenSoundscape ofrece opciones para poder manejar las anotaciones como crear un subset con pocas clases
classes_to_keep = ["WOTH", "HETH"]
thrush_annotations = annotations.subset(classes_to_keep)
thrush_annotations.df.annotation.value_counts()

## Preparar las anotaciones para que puedan ser incorporadas en el flujo de trabajo de entrenamiento-evaluacion de un clasificador supervisado de machine learning

Para que podamos usar las anotaciones tenemos que cambiar el formato a "multi-hot encoding labels" en donde cada fila es una anotacion y la presencia o ausencia de cada una de las clases incluidas es marcada con "1" y "0" en columnas independientes. El formato de Multi-hot encoding tambien facilita la representacion de anotaciones con mas de una clase por clip (multilabel)


In [65]:
# A veces se desean anotaciones mas cortas (Reminder para dibujar las cajas desde el tiempo inicial exacto de cada señal)
clip_duration = 3
clip_overlap = 0
min_label_overlap = 0.25
species_of_interest = ["NOCA", "EATO", "SCTA", "BAWW", "BCCH", "AMCR", "NOFL"]

labels_df = annotations.clip_labels(
    clip_duration=clip_duration,
    clip_overlap=clip_overlap,
    min_label_overlap=min_label_overlap,
    class_subset=species_of_interest,  # En caso de que quieras hacer un subset
)
labels_df.head()

NOCA  \
file                                               start_time end_time          
datos_anotados/Recordings/Recording_1/Recording... 0.0        3.0       False   
                                                   3.0        6.0       False   
                                                   6.0        9.0       False   
                                                   9.0        12.0      False   
                                                   12.0       15.0      False   

                                                                         EATO  \
file                                               start_time end_time          
datos_anotados/Recordings/Recording_1/Recording... 0.0        3.0        True   
                                                   3.0        6.0       False   
                                                   6.0        9.0        True   
                                                   9.0        12.0      False   
                                                   12.0       15.0      False   

                                                                         SCTA  \
file                                               start_time end_time          
datos_anotados/Recordings/Recording_1/Recording... 0.0        3.0       False   
                                                   3.0        6.0       False   
                                                   6.0        9.0       False   
                                                   9.0        12.0      False   
                                                   12.0       15.0      False   

                                                                         BAWW  \
file                                               start_time end_time          
datos_anotados/Recordings/Recording_1/Recording... 0.0        3.0       False   
                                                   3.0        6.0       False   
                                                   6.0        9.0       False   
                                                   9.0        12.0      False   
                                                   12.0       15.0      False   

                                                                         BCCH  \
file                                               start_time end_time          
datos_anotados/Recordings/Recording_1/Recording... 0.0        3.0       False   
                                                   3.0        6.0       False   
                                                   6.0        9.0       False   
                                                   9.0        12.0      False   
                                                   12.0       15.0      False   

                                                                         AMCR  \
file                                               start_time end_time          
datos_anotados/Recordings/Recording_1/Recording... 0.0        3.0       False   
                                                   3.0        6.0       False   
                                                   6.0        9.0       False   
                                                   9.0        12.0      False   
                                                   12.0       15.0       True   

                                                                         NOFL  
file                                               start_time end_time         
datos_anotados/Recordings/Recording_1/Recording... 0.0        3.0       False  
                                                   3.0        6.0       False  
                                                   6.0        9.0       False  
                                                   9.0        12.0      False  
                                                   12.0       15.0      False